In [1]:
# 14:25 begin 
import os, re
import xlutils
import databaker
import databaker.constants
from databaker.constants import *
import xypath
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, savepreviewhtmlBAGS, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [2]:
inputfile = "rftberd2014datasets_tcm77-424594.xls"
outputfile = "%s.csv" % os.path.splitext(inputfile)[0]
previewfile = "preview.html"


In [3]:
print("Loading %s which has size %d bytes" % (inputfile, os.path.getsize(inputfile)))
tableset = xypath.loader.table_set(inputfile, extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names", [tab.name  for tab in tabs])


Loading rftberd2014datasets_tcm77-424594.xls which has size 2217984 bytes
Table names ['Main Menu', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', 'Map 1', 'Map 2', '20', '21', '22', '23', '24', '25', '26', '27', '28', 'PGs']


In [4]:
title = tabs[0].excel_ref('B3').value
mtitle = re.match('Research and Development in UK Businesses, (\d\d\d\d) - Datasets', title)
assert mtitle, ("NoMatch", title)
year = mtitle.group(1)


In [5]:
tab = tabs[11]
savepreviewhtmlBAGS([tab.filter(contains_string('TOTAL')), tab.filter(contains_string('Pulp'))], previewfile)
tab.filter(contains_string('TOTAL')).shift(UP).assert_one()


opening file preview.html
table '11' written


{<B6 ''>}

In [13]:
conversionsegments = [ ]

for tab in tabs:
    tab_names = {
                '10':'Current and capital expenditure on R&D',
                '11':'Current expenditure on R&D',
                '12':'Extramural expenditure by R&D by UK Business',
                '13':'Sources of Funds for R&D in UK Business',
                '14':'Employment in R&D for UK Business',
                '16':'Civil and defence R&D expenditure in UK business',
                '24':'R&D Expenditure in UK business (UK or overseas ownership)',
                }
    if tab.name in tab_names:
        anchor = min(tab.filter(contains_string('TOTAL')), key=lambda c: (c.y, c.x)).shift((0,-2)) # tab.excel_ref('B5')
        obs = anchor.fill(RIGHT).fill(DOWN).is_not_blank().is_not_whitespace()     
        top = anchor.shift((0,-2)).fill(RIGHT).glue(lambda cell: cell.extrude(0,1))
        
        dimensions = [ (anchor.fill(DOWN).is_not_blank().is_not_whitespace(), 'Industry', DIRECTLY, LEFT),
                       (top, tab_names[tab.name], DIRECTLY, ABOVE)
                     ]

        obs = obs - tab.filter(contains_string('Source: Office for National Statistics')).expand(LEFT).expand(DOWN)
        obs = obs.is_number()
        conversionsegments.append((tab, dimensions, obs))
        
        assert min(ob.value  for ob in obs) > 10, "Observation out of range"
savepreviewhtml(conversionsegments[-1], batchcelllookup, previewfile)


AssertionError: Observation out of range

In [14]:
savepreviewhtml(conversionsegments[-1], batchcelllookup, previewfile)


opening file preview.html
table '10' written
javascript calculated


In [24]:
    
# this is the preview system
conversionsegment = conversionsegments[4]
savepreviewhtml(conversionsegment, batchcelllookup, previewfile)



opening file preview.html
table '14' written
javascript calculated


In [28]:
try:
    assert False, 11
except AssertionError as e:
    print(e)
    pass

11


In [ ]:
print("converting and writing %d conversion segments into %s" % (len(conversionsegments), outputfile))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment, batchcelllookup)
    convertedrows.append(rows)

# here we can do further sorting and substitution   

print("Finally writing the file", outputfile)
writetechnicalCSV(outputfile, conversionsegments, convertedrows)

